In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import datetime as dt
import sqlite3 as db
#import matplotlib.pyplot as pp

In [2]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [4]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [8]:
def weeklyAbundance():
    cnx = connectDB()
    try:
        ObsDataset = pd.read_sql('SELECT comName,locId,obsDt,howMany,subId FROM historicObservations WHERE comName in ("Canada Jay","Orange-crowned Warbler");',con=cnx,parse_dates=['obsDt']) #Canada Jay filter is for testing purposes only
        ObsDataset['obsDt_week'] = ObsDataset['obsDt'].dt.isocalendar().week
        #group up data
        ObsDataset = ObsDataset.groupby(['comName','locId','obsDt_week'])['howMany'].median()
        ObsDataset = pd.DataFrame(ObsDataset)
        ObsDataset['relativeAbundance'] = (ObsDataset['howMany']/ObsDataset['howMany'].mean())/max(ObsDataset['howMany']/ObsDataset['howMany'].mean())
        ObsDataset.sort_values(by=['comName','obsDt_week'],ascending=True,inplace=True)   #not needed
        ObsDataset.reset_index(inplace=True)   #not needed
    except Exception as calcEx:
        raise UserWarning(calcEx)
    finally: cnx.close()
    return ObsDataset

In [62]:
##TODO #71 apply polynomial interpolation to the weeklyAbundance values for use in forecasting --DONE

def forecastWklyAbd():
    realObsAbd = weeklyAbundance()
    try:
        keyList = realObsAbd.drop_duplicates(subset=['comName'])
        realObsAbd = realObsAbd.groupby(['comName','obsDt_week'])['relativeAbundance'].median()
        realObsAbd = realObsAbd.reset_index()
        forecastList = []
        for i in keyList.itertuples():
            allweek = pd.DataFrame({'obsDt_week':range(1,53)})
            obsAbd = realObsAbd[realObsAbd['comName'] == i.comName]
            obsAbd.drop(columns=['comName'],inplace=True)
            ptForxt = pd.merge(left=obsAbd,right=allweek,left_on='obsDt_week',right_on='obsDt_week',how='outer')
            ptForxt.set_index('obsDt_week',inplace=True)
            ptForxt.sort_index(axis='index',ascending=True,inplace=True)
            ptForxt.interpolate(method='polynomial',order=3,inplace=True,limit=3,limit_direction='both')
            ptForxt['comName'] = i.comName
            ptForxt = ptForxt.reset_index()
            forecastList.append(ptForxt)
        forecastAbd = pd.DataFrame()
        forecastAbd = pd.concat(forecastList,ignore_index=True)
    except Exception as abdForxExc:
        raise UserWarning(abdForxExc)
    return forecastAbd

In [63]:
forx = forecastWklyAbd()
forx

C:\Users\lukew\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,obsDt_week,relativeAbundance,comName
0,1,0.076923,Canada Jay
1,2,1.782393,Canada Jay
2,3,2.803381,Canada Jay
3,4,3.255086,Canada Jay
4,5,NaN,Canada Jay
...,...,...,...
100,48,NaN,Orange-crowned Warbler
101,49,NaN,Orange-crowned Warbler
102,50,NaN,Orange-crowned Warbler
103,51,NaN,Orange-crowned Warbler


In [34]:
trx = weeklyAbundance()
trx

,comName,locId,obsDt_week,howMany,relativeAbundance
0,Canada Jay,L8312096,1,1.0,0.076923
1,Canada Jay,L12172834,10,6.0,0.461538
2,Canada Jay,L12172834,11,2.0,0.153846
3,Canada Jay,L10129014,13,4.0,0.307692
4,Canada Jay,L12172834,40,7.0,0.538462
5,Canada Jay,L12172834,49,2.0,0.153846
6,Canada Jay,L8312096,49,1.0,0.076923
7,Canada Jay,L12172834,52,4.0,0.307692
8,Canada Jay,L424117,52,3.0,0.230769
9,Canada Jay,L8312096,52,1.0,0.076923


In [ ]:
##TODO #66 split out birds into quartiles based on the number of observations available and the YOY frequency. Results will determine the strength of forecasting required

In [90]:
##TODO #64 using the relative abundance value from weeklyAbundance(), write a function that returns FAO land coverage preference --DONE
##TODO #68 add seasons to relative abundance function, then add in a loop over each season in the function seasonalHabitatPreference
def seasonalHabitatPref():
    cnx = connectDB()
    try:
        wklyAbd = weeklyAbundance()
        #start season loop here
        rawList = []
        for locId in wklyAbd.itertuples():
            query = f'SELECT Barren,Urban,Grasslands,Savannas,waterBodies,evergreenNeedleleafForest,evergreenBroadleafForests,openForests,mixedBroadleafandNeedleleafForests,sparseForests,denseHerbaceous,sparseHerbaceous FROM FAO_by_locId WHERE FAO_by_locId.locId = "{locId.locId}"'
            lx = pd.read_sql(query,con=cnx)
            lx = lx.applymap(lambda x: locId.relativeAbundance*x,na_action='ignore')
            lx['comName'] = locId.comName
            lx['obsDt_week'] = locId.obsDt_week
            rawList.append(lx)
        results = pd.DataFrame()
        results = pd.concat(rawList,ignore_index=True)
        results = results.groupby(['comName','obsDt_week']).mean()
        results.reset_index(inplace=True)
        #apply polynomial interpolation of observed habitat preferences
        results = pd.DataFrame(results)
    except Exception as prefExc:
        raise UserWarning(prefExc)
    finally: cnx.close()
    return results

In [91]:
adun = seasonalHabitatPref()
adun


,comName,obsDt_week,Urban,Savannas,evergreenNeedleleafForest,evergreenBroadleafForests,openForests,mixedBroadleafandNeedleleafForests,sparseForests
0,Canada Jay,1,NaN,NaN,0.010000,NaN,NaN,0.067692,NaN
1,Canada Jay,10,NaN,NaN,0.461538,NaN,NaN,NaN,NaN
2,Canada Jay,11,NaN,NaN,0.153846,NaN,NaN,NaN,NaN
3,Canada Jay,13,0.24,NaN,0.067692,NaN,0.058462,NaN,0.181538
4,Canada Jay,40,NaN,NaN,0.538462,NaN,NaN,NaN,NaN
5,Canada Jay,49,NaN,NaN,0.081923,NaN,NaN,0.067692,NaN
6,Canada Jay,52,NaN,NaN,0.117436,0.030000,NaN,0.127308,NaN
7,Canada Jay,53,NaN,NaN,0.615385,NaN,NaN,NaN,NaN
8,Orange-crowned Warbler,16,NaN,NaN,0.057692,NaN,NaN,0.019231,NaN
9,Orange-crowned Warbler,17,NaN,NaN,0.035769,0.009231,NaN,0.078846,NaN


In [ ]:
##TODO #65 using historic habitat preference data, forecast the seasonal habitat preference for each species of bird